# Functions as Objects

> Functions in Python are **first-class objects.** Programming language researchers define a “first-class object” as a program entity that can be:

- **Created at runtime**
- **Assigned to a variable or element in a data structure**
- **Passed as an argument to or Returned as the result of a function**

In [2]:
def factororial(n):
    """Calculate the factororial of a number."""
    return 1 if n == 0 else n * factororial(n - 1)


print(factororial(5))

factororial.__doc__

120


'Calculate the factororial of a number.'

## Anonymous Functions

In [5]:
fruits = ["strawberry", "fig", "apple", "cherry", "raspberry", "banana"]
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## Function Parameters
### Keyword-Only Arguments

In [8]:
def tag(name, *content, class_=None, **attrs):
    """Generate one or more HTML tags"""
    if class_ is not None:
        attrs["class"] = class_
    attr_pairs = (f' {attr}="{value}"' for attr, value in sorted(attrs.items()))
    attr_str = "".join(attr_pairs)
    if content:
        elements = (f"<{name}{attr_str}>{c}</{name}>" for c in content)
        return "\n".join(elements)
    else:
        return f"<{name}{attr_str} />"


tag("p", "hello")

'<p>hello</p>'

### Postional-Only Arguments

In [9]:
def divmod(a, b, /):
    return (a // b, a % b)


divmod(10, 3)

(3, 1)

### freezing parameters

In [14]:
from functools import partial
from operator import mul


def foo(a: int, b: int, c: int) -> int:
    print(a, b, c)
    return a * b * c


mul_and_times_3 = partial(foo, c=3)
mul_and_times_3(4, 5)

4 5 3


60

## Decorators and Closures

In [15]:
def decorate(func):
    def wrapper(*args, **kwargs):
        print(func.__name__)
        return func(*args, **kwargs)

    return wrapper


@decorate
def target():
    print("running target()")


target()

target
running target()


A key feature of decorators is that they run right after the decorated function is defined. 

In [17]:
registry = []


def register(func):
    print(f"running register({func})")
    registry.append(func)
    return func


@register
def f1():
    print("running f1()")


@register
def f2():
    print("running f2()")


def f3():
    print("running f3()")


def main():
    print("running main()")
    print("registry ->", registry)
    f1()
    f2()
    f3()


main()

running register(<function f1 at 0x104ac6160>)
running register(<function f2 at 0x104ac6020>)
running main()
registry -> [<function f1 at 0x104ac6160>, <function f2 at 0x104ac6020>]
running f1()
running f2()
running f3()


### Closures

> A closure is a function that  retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

In [20]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)

    return averager


def make_averager_nonlocal():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager


avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


### Decorators in STL

Python has three built-in functions that are designed to decorate methods:

- `property`
- `classmethod`
- `staticmethod`

In [26]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        return self._radius

    @radius.setter
    def radius(self, value):
        if value < 0:
            raise ValueError("Radius cannot be negative")
        self._radius = value

    @property
    def area(self):
        return 3.14 * self._radius**2


c = Circle(4.0)
c.radius = 5
print(c.area)
print(c.radius)

78.5
5


In [30]:
class Employee:
    num_employees = 0

    def __init__(self, name):
        self.name = name
        Employee.num_employees += 1

    @classmethod
    def from_string(cls, string):
        first, last = string.split()
        return cls(f"{first} {last}")

    @classmethod
    def get_count(cls):
        return cls.num_employees


emp = Employee.from_string("Alice Smith")
print(Employee.get_count())

1


In [21]:
import functools


@functools.cache
def fib(n):
    return 1 if n < 2 else fib(n - 1) + fib(n - 2)


fib(100)

573147844013817084101